In [ ]:
import os
import pandas as pd
import numpy as np
from os import listdir
from os import path
import json
import matplotlib.pyplot as plt
import re
# % matplotlib inline

#  file threshold

In [ ]:
# parse json to files and actions

def parse_json(filepath_list):
    '''
    function used to parse json of each commit json file
    
    Args:
        filepath_list - list of filepaths
    
    Returns:
        files_json - list object contains parsed information
    
    '''
    number_of_files = []
    files_json = []
    
    # each commits
    for path in filepath_list:
        with open(path,'rb') as f:
            data = json.load(f)
            files_list = []
            # each file in commits
            for file in data['files']:
                # parse only cluster file
                for key in file.keys():
                    if re.match('^.*_cluster$',key):
                        actions_list = []
                        actions = file[key]['actions']
                        # each action in file
                        for action in actions:
                            actions_list.append(action['root'])
                        files_list.append(actions_list)
        
        files_json.append(files_list)        
    
    # return
    return files_json

# get file path list
filepath_list = ['../data/tmp_JSON_labeled_commits/' + path for path in listdir('../data/tmp_JSON_labeled_commits/')]
files_json = parse_json(filepath_list)

In [ ]:
# getting file threshold
threshold = 0.95
number_of_files = [len(files) for files in files_json]

def get_file_threshold(number_of_files, threshold = 0.95):
    '''
    get padding threshold for files dimension
    
    Args:
        number_of_files - array of the number of files in each commits
        threshold - drop all commits with its the number of files beyond this threshold
    Returns:
        padding threshold - number
    '''
    
    total_files = len(number_of_files)
    number_of_files = np.array(number_of_files)
    bincount = np.bincount(number_of_files)

    sum_file = 0
    for index, item in enumerate(bincount):
        sum_file += item
        #print(index,item)
        #print(sum_file)
        if sum_file > threshold*total_files:
            padding_files_threshold = index
            break
            
    return padding_files_threshold

file_threshold = get_file_threshold(number_of_files, threshold)
print(file_threshold)

# root_threshold

In [ ]:
threshold = 0.95
number_of_root = []
root_count = []
for files in files_json:
    number_of_root.extend([len(roots) for roots in files])
    root_count.append([len(roots) for roots in files])

In [ ]:
# getting file threshold

def get_root_threshold(number_of_root, threshold = 0.95):
    '''
    get padding threshold for root dimension
    
    Args:
        number_of_root - array of the number of root in each commits
        threshold - drop all commits with its the number of roots beyond this threshold
    Returns:
        padding threshold - number
    '''
    
    total_root = len(number_of_root)
    number_of_root = np.array(number_of_root)
    bincount = np.bincount(number_of_root)

    sum_root = 0
    for index, item in enumerate(bincount):
        sum_root += item
        #print(index,item)
        #print(sum_file)
        if sum_root > threshold*total_root:
            padding_root_threshold = index
            break
            
    return padding_root_threshold

root_threshold = get_root_threshold(number_of_root, threshold)
print(root_threshold)

# select files which comply with the standard  
[roots number $\le$ 38, files number $\le$ 22]

In [ ]:
cmt_used = [] # index of commits which comply with the standard.
i = 0
for counts in root_count:
    if (number_of_files[i]<=22) and (False not in [items <=38 for items in counts]):
        cmt_used.append(i)
    i += 1


In [ ]:
cmt_used

In [ ]:
print("We will use {} commits while the total number of commits is {}".format(
    len(cmt_used), len(files_json)))